In [1]:
import geopandas as gpd
import pandas as pd
import json
import folium

# Load GeoJSON file
gdf_neighborhoods = gpd.read_file('philadelphia.geojson')

# Load CSV data
data = pd.read_csv('yelp_academic_dataset_with_price_rating.csv')

# Convert DataFrame to GeoDataFrame
gdf_data = gpd.GeoDataFrame(
    data, 
    geometry=gpd.points_from_xy(data.longitude, data.latitude),
    crs="EPSG:4326"  # Make sure this matches your GeoJSON CRS
)

# Spatial join - points to polygons
data_with_neighborhoods = gpd.sjoin(gdf_data, gdf_neighborhoods, how="inner", op='intersects')

# Calculate average review count and restaurant count per neighborhood
data_grouped = data_with_neighborhoods.groupby('name_right').agg(
    average_review_count=('review_count', 'mean'),
    restaurant_count=('review_count', 'count')
).reset_index()

data_grouped['average_review_count'] = data_grouped['average_review_count'].round(0)

# Load your GeoJSON as a GeoDataFrame
gdf = gpd.read_file('philadelphia.geojson')

# Join your DataFrame with the GeoDataFrame
gdf = gdf.set_index('name').join(data_grouped.set_index('name_right'), how="left").reset_index()

# Convert all Timestamp columns to strings
for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, utc]']).columns:
    gdf[col] = gdf[col].astype(str)

# Save back to GeoJSON
merged_geojson = json.loads(gdf.to_json())

# Create a map centered around Philadelphia
m = folium.Map(location=[39.9526, -75.1652], zoom_start=12)

# Add the choropleth layer
folium.Choropleth(
    geo_data=merged_geojson,
    name='choropleth',
    data=gdf,
    columns=['name', 'average_review_count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',  # Color scheme: can be adjusted
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Review Count per Neighborhood'
).add_to(m)

# Add tooltips using the merged GeoJSON
folium.GeoJson(
    data=merged_geojson,
    style_function=lambda feature: {'color': 'black', 'fillColor': '#ffff00', 'weight': 0.5, 'fillOpacity': 0.2},
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'average_review_count', 'restaurant_count'],
        aliases=['Neighborhood:', 'Average Reviews:', 'Restaurant Count:'],
        style="background-color: #F0EFEF; border: 2px solid black; border-radius: 3px; box-shadow: 3px;",
        max_width=250,
    )
).add_to(m)

# Add a layer control to toggle on/off
folium.LayerControl().add_to(m)

# Save the map to an HTML file
map_file = 'philadelphia_neighborhoods_map2.html'
m.save(map_file)
print(f"Map has been saved to: {map_file}")


Map has been saved to: philadelphia_neighborhoods_map2.html


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [2]:
import geopandas as gpd
import pandas as pd
import json

# Load GeoJSON file
gdf_neighborhoods = gpd.read_file('philadelphia.geojson')

# Load CSV data
data = pd.read_csv('yelp_academic_dataset_with_price_rating.csv')

# Convert DataFrame to GeoDataFrame
gdf_data = gpd.GeoDataFrame(
    data, 
    geometry=gpd.points_from_xy(data.longitude, data.latitude),
    crs="EPSG:4326"  # Make sure this matches your GeoJSON CRS
)

# Spatial join - points to polygons
data_with_neighborhoods = gpd.sjoin(gdf_data, gdf_neighborhoods, how="inner", op='intersects')

# Calculate average review count and restaurant count per neighborhood
data_grouped = data_with_neighborhoods.groupby('name_right').agg(
    average_review_count=('review_count', 'mean'),
    restaurant_count=('review_count', 'count')
).reset_index()

data_grouped['average_review_count'] = data_grouped['average_review_count'].round(0)

# Load your GeoJSON as a GeoDataFrame
gdf = gpd.read_file('philadelphia.geojson')

# Join your DataFrame with the GeoDataFrame
gdf = gdf.set_index('name').join(data_grouped.set_index('name_right'), how="left").reset_index()

# Convert all Timestamp columns to strings
for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, utc]']).columns:
    gdf[col] = gdf[col].astype(str)

# Save back to GeoJSON
merged_geojson = gdf.to_json()

# Save the merged GeoJSON to a file
with open('merged_geojson.json', 'w') as f:
    json.dump(json.loads(merged_geojson), f)

print("Merged GeoJSON has been saved to: merged_geojson.json")


Merged GeoJSON has been saved to: merged_geojson.json


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
